https://news.adobe.com/news/default.aspx

In [1]:
import pandas as pd,requests,bs4
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
titles,thumbnails,hrefs = [],[],[]

range_count = 2023

for x in range(2021,range_count):
    try:
        api = f"https://news.adobe.com/feed/PressRelease.svc/GetPressReleaseList?apiKey=BF185719B0464B3CB809D23926182246&LanguageId=1&bodyType=0&pressReleaseDateFilter=3&categoryId=1cb807d2-208f-4bc3-9133-6a9ad45ac3b0&pageSize=-1&pageNumber=0&tagList=&includeTags=true&year={x}&excludeSelection=1"
        req = requests.get(api)
        json = req.json()
        try:
            for i in range(100):
                data = json['GetPressReleaseListResult'][i]
                title = data["Headline"]
                titles.append(title)
                href = "https://news.adobe.com/" + data["LinkToDetailPage"]
                hrefs.append(href)
                thumbnail = data["ThumbnailPath"]
                thumbnails.append(thumbnail)
        except:
            pass
    except:
        pass
    print(str(range_count-(x+1)) + " pages remaining..",end="\r")

In [3]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [4]:
dates,articles = [],[]
range_count = len(href)

for number,link in enumerate(hrefs[:range_count]):
    soup = parse_webpage_bs(link)

    date_containers = soup.findAll('div',{'class' :'module_date-time'})

    for d in date_containers:
        date = (d.text).replace("\n","")
        dates.append(date)
    
    art_containers = soup.findAll('div',{'class' :'module_body'})

    for art in art_containers:
        article = art.text
        articles.append(article)

    print(str(len(hrefs)-(number+1)) + " url remaining..",end="\r")

In [5]:
#zippling the list so that it'll form the tuple
zipped = list(zip(dates,titles,articles,hrefs,thumbnails))
#creating dataframe 
temp_df = pd.DataFrame(zipped,columns=['date','title','article','url','thumbnail'])
temp_df.head()

,date,title,article,url,thumbnail
0,"Thursday, December 16, 2021 08:05 AM",Adobe Reports Record Q4 and Fiscal 2021 Revenu...,\n\nCompany reports first $3 billion Digital M...,https://news.adobe.com//news/news-details/2021...,https://s23.q4cdn.com/979560357/files/Newsroom...
1,"Thursday, December 16, 2021 08:05 AM",Adobe Announces Changes to Executive Leadershi...,"\nSAN JOSE, Calif.--(BUSINESS WIRE)-- \r\nAdob...",https://news.adobe.com//news/news-details/2021...,
2,"Wednesday, December 15, 2021 04:45 PM",Adobe Named a Leader in World’s First Major Ev...,\nIDC MarketScape: Worldwide Customer Data Pla...,https://news.adobe.com//news/news-details/2021...,https://s23.q4cdn.com/979560357/files/NR-Heade...
3,"Monday, December 13, 2021 11:00 AM",Adobe Introduces Creative Cloud Express,\n\n\nNew unified web and mobile product empow...,https://news.adobe.com//news/news-details/2021...,https://s23.q4cdn.com/979560357/files/AdobeNew...
4,"Thursday, December 09, 2021 08:00 AM",Adobe Digital Price Index: Online Inflation Hi...,\n\n\nOnline prices increased 3.5% year-over-y...,https://news.adobe.com//news/news-details/2021...,https://s23.q4cdn.com/979560357/files/NR-Heade...


In [6]:
#to csv
temp_df.to_csv('adobe news.csv')

#to json
temp_df.to_json('adobe news.json')